AMT5005MX Python Project AppleOrangeAI

In [68]:
pip install torch


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [69]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [70]:
#Necessary pytorch libaries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

In [71]:
#Defining a few hyperparamaters for later
num_classes = 6 #- ‘num_classes’: The number of classes in our new classification task.
batch_size = 32 #- ‘batch_size’: Number of images processed at a time. A typical value is 32 or 64.
num_epochs = 1 #- ‘num_epochs’: Number of complete passes through the training dataset. Here we use 1 for demonstration, but in practice, you may use 10, 20, or more.
learning_rate = 0.001 #- ‘learning_rate’: The rate at which the model updates its weights during training.

In [72]:
#Data Preprocessing

data_transforms = {
 'train': transforms.Compose([
 transforms.Resize((224, 224)), #all images are resized to VVG16, fixing outliers and making the images optimal for VGG16
 transforms.RandomHorizontalFlip(), #Randomly flips the images horizontally, this effectively allows for more training and test material by doubling the number of available images (flipped and unflipped)
 transforms.ToTensor(),
 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) #Normalizes the pixel values (mandatory for using my pre-trained model)
 ]),
 'val': transforms.Compose([
 transforms.Resize((224, 224)),
 transforms.ToTensor(),
 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
 ]),
}

In [73]:
#Change these paths to your own to avoid errors
root_dir = './apple_orange'

In [74]:
#Loads the training and validation datasets using 'ImageFolder' for later use
train_dataset = datasets.ImageFolder(root=root_dir, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(root=root_dir, transform=data_transforms['val'])

In [75]:
#creates data loaders to fectch data in batches
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [76]:
#loads the Pre-Trained VGG16 Model
model = models.vgg16(pretrained=True)

c:\Users\Myles\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Myles\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [77]:
#Freezes unnecessary parts of the model to save resources
for param in model.features.parameters():
 param.requires_grad = False

In [78]:
#modifies the final layer to match the number of classes in our task
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

In [79]:
#creates a Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate)

Model Training

In [80]:
# Check if CUDA is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Using device: {device}")

Using device: cpu


In [81]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [83]:
for epoch in range(num_epochs):
   model.train()
   running_loss = 0.0
   for inputs, labels in train_loader:
     inputs, labels = inputs.to(device), labels.to(device)
     optimizer.zero_grad()
     outputs = model(inputs)
     loss = criterion(outputs, labels)
     loss.backward()
     optimizer.step()
     running_loss += loss.item() * inputs.size(0)
   epoch_loss = running_loss / len(train_loader.dataset)
   print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

Epoch 1/1, Loss: 1.1274


Model Evaluation

In [84]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = correct / total
    print(f'Validation Accuracy: {val_accuracy:.4f}')

Validation Accuracy: 0.6713


In [85]:
#saves the trained model
torch.save(model.state_dict(), 'vgg16_finetuned.pth')

In [87]:
# Load the saved state dictionary
state_dict = torch.load('vgg16_finetuned.pth')

# Load the state dictionary into the model
model.load_state_dict(state_dict)

<All keys matched successfully>